# Listing all events in cv

In [1]:
import cv2 as cv
import numpy as np

events = [i for i in dir(cv) if 'EVENT' in i]
print( np.array(events) )

['EVENT_FLAG_ALTKEY' 'EVENT_FLAG_CTRLKEY' 'EVENT_FLAG_LBUTTON'
 'EVENT_FLAG_MBUTTON' 'EVENT_FLAG_RBUTTON' 'EVENT_FLAG_SHIFTKEY'
 'EVENT_LBUTTONDBLCLK' 'EVENT_LBUTTONDOWN' 'EVENT_LBUTTONUP'
 'EVENT_MBUTTONDBLCLK' 'EVENT_MBUTTONDOWN' 'EVENT_MBUTTONUP'
 'EVENT_MOUSEHWHEEL' 'EVENT_MOUSEMOVE' 'EVENT_MOUSEWHEEL'
 'EVENT_RBUTTONDBLCLK' 'EVENT_RBUTTONDOWN' 'EVENT_RBUTTONUP']


# Global Variables

In [2]:
isMouseDown = False
selectionPoints = [(), ()]
windowName = "Image"
windowImage = None

## Some Functions

In [3]:
def cropping(AreaPoints):
    global windowImage, windowName
    windowImage = windowImage[AreaPoints[0][1]:AreaPoints[1][1] , AreaPoints[0][0]:AreaPoints[1][0]]
    cv.imshow(windowName , windowImage)

def Rectangle(AreaPoints):
    global windowImage, windowName
    windowImage = cv.rectangle(windowImage,AreaPoints[0],AreaPoints[1],(0,255,0),3)
    cv.imshow(windowName , windowImage)

def Circle(AreaPoints):
    global windowImage, windowName
    windowImage = cv.circle(windowImage,AreaPoints[0],int(distance(AreaPoints[0] , AreaPoints[1])),(0,255,0),3)
    cv.imshow(windowName , windowImage)

def mask(AreaPoints):
    global windowImage, windowName
    x , y = AreaPoints[0]
    color = windowImage[y , x]
    hsv_image = cv.cvtColor(windowImage , cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv_image, *colorRange(color))
    threshold = cv.bitwise_and(windowImage, windowImage, mask=mask)
    cv.imshow( "color selection", threshold)

def colorRange(bgr_color):
    bgr_color = np.array([[bgr_color]])
    hsv_color = cv.cvtColor(bgr_color, cv.COLOR_BGR2HSV)
    color_hue = int(hsv_color[0][0][0])
    color_saturation = int(hsv_color[0][0][1])
    color_value = int(hsv_color[0][0][2])
    lower_bound = color_hue - 10, (color_saturation - 50 if color_saturation >= 50 else 0 ) , (color_value - 50 if color_value >= 50 else 0)

    upper_bound = color_hue + 10, (color_saturation + 50 if color_saturation <= 205 else 255 ) , (color_value + 50 if color_value <= 205 else 255)
    return lower_bound , upper_bound


def distance(point1 , point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def mouseSelectedArea(functionOnSelectedArea , min_distance = 5 , onMouseDownOnly = False):
    def func(event , x , y , flags , param):
        global isMouseDown, windowImage, selectionPoints, windowName
        if event == cv.EVENT_LBUTTONDOWN:
            selectionPoints[0] = (x , y)
            isMouseDown = True
            if(onMouseDownOnly):
                functionOnSelectedArea(selectionPoints)
                isMouseDown = False

        if event == cv.EVENT_LBUTTONUP and isMouseDown and not onMouseDownOnly:
            selectionPoints[1] = (x , y)
            if(distance(selectionPoints[0] , selectionPoints[1]) < min_distance):
                return
            functionOnSelectedArea(selectionPoints)
            isMouseDown = False
    return func

In [4]:
import numpy as np
import cv2 as cv
original_img = cv.imread("bunny.jpg")

# # will not use it because the image is read in RGB on my machine
# original_img = cv.cvtColor(original_img ,  cv.COLOR_BGR2RGB)

dst_img  = cv.resize(original_img, None , fx=0.1,fy = 0.1,interpolation=cv.INTER_AREA)

windowImage = dst_img
windowName = "bunny.jpg"

startImage = dst_img.copy()
font = cv.FONT_HERSHEY_COMPLEX
cv.putText(startImage,'press one character then select Area',(100,100), font, 1,(255,0,0),2,cv.LINE_AA)
cv.putText(startImage,'[r - c - u - m]',(300,150), font, 1,(255,0,0),2,cv.LINE_AA)
cv.putText(startImage,'press s to save',(280,200), font, 1,(0,0,255),2,cv.LINE_AA)

cv.imshow(windowName , startImage)

methods = {
            "r" : "rectangle",
            "c": "circle",
            "u":"crop",
            "m" : "mask"
           }

while True:
    if(windowImage.shape != startImage.shape):
        cv.imshow(windowName , windowImage)
    else :
        cv.imshow(windowName , startImage)
    waitkey = cv.waitKey(0)
    if  waitkey & 0xFF == ord('s'):
        cv.imwrite("cropped.jpg" , windowImage)
        break
    if waitkey > 0 and chr(waitkey) in methods:
        startImage = windowImage
        selectedMethod = methods[chr(waitkey)]
        if selectedMethod == "crop":
            print("crop selected")
            cv.setMouseCallback(windowName , mouseSelectedArea(cropping))
        elif selectedMethod == "rectangle":
            print("rectangle selected")
            cv.setMouseCallback(windowName , mouseSelectedArea(Rectangle))
        elif selectedMethod == "circle":
            print("circle selected")
            cv.setMouseCallback(windowName , mouseSelectedArea(Circle))
        elif selectedMethod == "mask":
            print("mask selected")
            cv.setMouseCallback(windowName , mouseSelectedArea(mask , min_distance=-1))
    if waitkey & 0xFF == 27:
        break
    if waitkey & 0xFF == -1 & 0xFF:
        break
    
cv.destroyAllWindows()

mask selected
